###TORONTO CLUSTERING ASSIGNMENT


In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.16.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00 502.02 kB/s
geopy-1.16.0-p 100% |################################| Time: 0:00:00 768.20 kB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py_0 conda-forge
    branca:  0.3.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge

branca-0.3.0-p 100% |################################| Time: 0:00:00 804.03 kB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00 926.84 kB/s
altair-2.2.2-p 100% |###########################

###Build the pandas dataframe with all the required data of Toronto

In [2]:
#get url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
web_html = requests.get(url).text
soup = bs(web_html,'lxml')

#retrieve the table of the wiki article
table_data = soup.find('table')

table_b = table_data.find_all('b')
missing_neigh = table_b[7].text

table_p = table_data.find_all('p') 


#Set the values of Postal Code, Neighborhood and Borough into lists
borough_list = []
pc_list = []
neig_list = []

for i in range(len(table_p)):
    try:
        dis = table_p[i].a.text
        postal_code = table_p[i].b.text
        span_tag = table_p[i].span.text
        neighborhood = re.findall('\((.*)\)', span_tag)
        #if neighborhood == None: neighborhood = 'None'
        
        borough_list.append(dis)
        pc_list.append(postal_code)
        neig_list.append(neighborhood)
    except:
        continue

d = len(borough_list)
pc = len(pc_list)

#retrieve an untagged neighborhood 
neig_list[4] = [missing_neigh]

#clean the neighborhood list from simbols, and replace them with commas
neighb = []
for i in range(len(neig_list)):
    n = neig_list[i][0]
    x = n.replace('/',',')
    #print(x)
    neighb.append(x)
    
neighb1=[]    
for i in range(len(neig_list)):
    n1 = neighb[i]
    x1 = n1.replace('(',',')
    neighb1.append(x1)

neighb2=[]    
for i in range(len(neig_list)):
    n2 = neighb1[i]
    x2 = n2.replace(')',',')
    neighb2.append(x2)

#Create the data frame with the data
df_toronto = pd.DataFrame()
df_toronto['Postal Code'] = pc_list
df_toronto['Borough'] = borough_list
df_toronto['Neighborhood'] = neighb2

lat = [43.7532586, 43.72588229999999, 43.6542599, 43.718518, 43.6623015, 43.6678556, 43.8066863, 43.7459058, 43.7063972, 43.6571618, 43.709577, 43.65094320000001, 43.78453510000001, 43.7258997, 43.6953439, 43.6514939, 43.6937813, 43.6435152, 43.7635726, 43.6763574, 43.6447708, 43.68902560000001, 43.7709921, 43.7090604, 43.6579524, 43.669542, 43.773136, 43.8037622, 43.7543283, 43.7053689, 43.65057119999999, 43.6690051, 43.7447342, 43.7785175, 43.7679803, 43.685347, 43.6408157, 43.6479267, 43.7279292, 43.7869473, 43.7374732, 43.6795571, 43.6471768, 43.6368472, 43.7111117, 43.7574902, 43.7390146, 43.6689985, 43.6481985, 43.7137562, 43.7563033, 43.716316, 43.789053, 43.7284964, 43.7332825, 43.6911158, 43.7247659, 43.692657, 43.7701199, 43.7616313, 43.7280205, 43.6731853, 43.706876, 43.7574096, 43.7527583, 43.7127511, 43.6969476, 43.6616083, 43.696319, 43.7500715, 43.7827364, 43.7153834, 43.6727097, 43.6489597, 43.6369656, 43.6889054, 43.7942003, 43.7043244, 43.6626956, 43.6515706, 43.7816375, 43.6895743, 43.6532057, 43.8152522, 43.6864123, 43.6289467, 43.6056466, 43.7394164, 43.7995252, 43.6795626, 43.6464352, 43.6024137, 43.7067483, 43.8361247, 43.667967, 43.6484292, 43.6536536, 43.6658599, 43.6627439, 43.6362579, 43.6288408] 
lng = [-79.3296565, -79.3155716, -79.36063589999999, -79.4647633, -79.3894938, -79.5322424, -79.1943534, -79.352188, -79.30993699999999, -79.3789371, -79.4450726, -79.5547244, -79.1604971, -79.34092299999999, -79.3183887, -79.3754179, -79.4281914, -79.5772008, -79.1887115, -79.2930312, -79.37330639999999, -79.453512, -79.2169174, -79.3634517, -79.3873826, -79.4225637, -79.23947609999999, -79.3634517, -79.4422593, -79.3493719, -79.3845675, -79.4422593, -79.23947609999999, -79.3465557, -79.4872619, -79.3381065, -79.3817523, -79.4197497, -79.2620294, -79.385975, -79.4647633, -79.352188, -79.3815764, -79.4281914, -79.2845772, -79.37471409999999, -79.5069436, -79.3155716, -79.3798169, -79.49007379999999, -79.56596329999999, -79.23947609999999, -79.40849279999999, -79.4956974, -79.4197497, -79.47601329999999, -79.5322424, -79.2648481, -79.40849279999999, -79.5209994, -79.3887901, -79.4872619, -79.5181884, -79.273304, -79.40004929999999, -79.3901975, -79.4113072, -79.4647633, -79.5322424, -79.2958491, -79.4422593, -79.4056784, -79.4056784, -79.45632499999999, -79.615819, -79.5547244, -79.2620294, -79.3887901, -79.40004929999999, -79.4844499, -79.3043021, -79.3831599, -79.40004929999999, -79.2845772, -79.40004929999999, -79.3944199, -79.5013207, -79.58843689999999, -79.3183887, -79.3775294, -79.37484599999999, -79.5434841, -79.59405439999999, -79.20563609999999, -79.3676753, -79.3822802, -79.5069436, -79.3831599, -79.321558, -79.49850909999999, -79.5209994]

df_toronto['Latitude']=lat
df_toronto['Longitude']=lng
df_toronto.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


###Create a map of Toronto

In [4]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:138: UserWarning: Using Nominatim with the default "geopy/1.16.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [5]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

###create a new data frame with and the neighborhoods in Scarborough 

In [1]:
# The code was removed by Watson Studio for sharing.

In [7]:
df_toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [8]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848


###Create a map of Scarborough and its neighborhoods

In [9]:
address1 = 'Scarborough,Toronto'

geolocator1 = Nominatim()
location1 = geolocator1.geocode(address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude1, longitude1))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:138: UserWarning: Using Nominatim with the default "geopy/1.16.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [10]:
map_scarb = folium.Map(location=[latitude1, longitude1], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarb)  
    
map_scarb

###Get the top 100 venues in the neighborhood 'Malvern', from Scarborough

In [11]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern , Rouge are 43.8066863, -79.1943534.


In [12]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, radius, LIMIT)

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5b7ef4d6db04f52627b92559'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-5085ec39e4b0b1ead2eb0818-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/toys_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1f3941735',
         'name': 'Toy / Game Store',
         'pluralName': 'Toy / Game Stores',
         'primary': True,
         'shortName': 'Toys & Games'}],
       'id': '5085ec39e4b0b1ead2eb0818',
       'location': {'address': '300 Borough Drive',
        'cc': 'CA',
        'city': 'Scarborough',
        'country': 'Canada',
        'crossStreet': 'in Scarborough Town Centre',
        'distance': 284,
        'formattedAddress': ['300 Borough Drive (in Scarborough Town Centre)',
         'Scarborough ON M1P 4P5

In [14]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(12)

,name,categories,lat,lng
0,Disney store,Toy / Game Store,43.775537,-79.256833
1,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
2,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
3,DAVIDsTEA,Tea Room,43.776613,-79.258516
4,SEPHORA,Cosmetics Shop,43.775592,-79.258242
5,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
6,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
7,Hot Topic,Clothing Store,43.775450,-79.257929
8,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
9,Tim Hortons,Coffee Shop,43.769936,-79.260368


In [16]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

40 venues were returned by Foursquare.


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

###Get the venues for each neighborhood in Scarborough

In [18]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Malvern , Rouge
Rouge Hill , Port Union , Highland Creek
Guildwood , Morningside , West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park , Ionview , East Birchmount Park
Golden Mile , Clairlea , Oakridge
Cliffside , Cliffcrest , Scarborough Village West
Birch Cliff , Cliffside West
Dorset Park , Wexford Heights , Scarborough Town Centre
Wexford , Maryvale
Agincourt
Clarks Corners , Tam O'Shanter , Sullivan
Milliken , Agincourt North , Steeles East , L'Amoreaux East
Steeles West , L'Amoreaux West
Upper Rouge


In [86]:
print(scarborough_venues.shape)
scarborough_venues

(82, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern , Rouge",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Swiss Chalet,43.767697,-79.189914,BBQ Joint
3,"Guildwood , Morningside , West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
5,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764042,-79.193371,Rental Car Location
6,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
7,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot
8,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop
9,Woburn,43.770992,-79.216917,Tim Hortons,43.770827,-79.223078,Coffee Shop


In [21]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff , Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clarks Corners , Tam O'Shanter , Sullivan",11,11,11,11,11,11
"Cliffside , Cliffcrest , Scarborough Village West",3,3,3,3,3,3
"Dorset Park , Wexford Heights , Scarborough Town Centre",7,7,7,7,7,7
"Golden Mile , Clairlea , Oakridge",9,9,9,9,9,9
"Guildwood , Morningside , West Hill",6,6,6,6,6,6
"Kennedy Park , Ionview , East Birchmount Park",5,5,5,5,5,5


In [22]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 52 uniques categories.


In [23]:
# one hot encoding
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Cosmetics Shop,Department Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,"Malvern , Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill , Port Union , Highland Creek",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood , Morningside , West Hill",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood , Morningside , West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood , Morningside , West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
scarb_onehot.shape

(82, 53)

In [71]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Cosmetics Shop,Department Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,Agincourt,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.25,0.0,0.000000,0.000000,0.000000,0.0,0.000000
1,"Birch Cliff , Cliffside West",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.25,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.25,0.0,0.000000,0.000000,0.000000,0.0,0.000000
2,Cedarbrae,0.000000,0.142857,0.0,0.000000,0.142857,0.142857,0.0,0.000000,0.000000,0.000000,0.00,0.142857,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.142857,0.00,0.000000,0.142857,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.142857,0.000000,0.0,0.000000
3,"Clarks Corners , Tam O'Shanter , Sullivan",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.090909,0.090909,0.00,0.000000,0.0,0.000000,0.090909,0.090909,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.090909,0.0,0.000000,0.090909,0.181818,0.0,0.090909,0.000000,0.00,0.0,0.000000,0.090909,0.000000,0.0,0.000000
4,"Cliffside , Cliffcrest , Scarborough Village West",0.333333,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.333333,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.000000
5,"Dorset Park , Wexford Heights , Scarborough To...",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.142857,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.285714,0.000000,0.000000,0.000000,0.000000,0.142857,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.142857,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.142857,0.0,0.142857
6,"Golden Mile , Clairlea , Oakridge",0.000000,0.000000,0.0,0.000000,0.222222,0.000000,0.0,0.000000,0.222222,0.111111,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.111111,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.111111,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.111111,0.000000,0.000000,0.0,0.000000
7,"Guildwood , Morningside , West Hill",0.000000,0.000000,0.0,0.166667,0.000000,0.000000,0.0,0.166667,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,

###Get top 10 venues per neighborhood

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Sandwich Place,Lounge,Breakfast Spot,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
1,"Birch Cliff , Cliffside West",General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant
2,Cedarbrae,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Hakka Restaurant,Bakery,Bank,Fried Chicken Joint,Coffee Shop,Grocery Store,General Entertainment
3,"Clarks Corners , Tam O'Shanter , Sullivan",Pizza Place,Noodle House,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Italian Restaurant,Rental Car Location,Pharmacy,Coffee Shop,Chinese Restaurant
4,"Cliffside , Cliffcrest , Scarborough Village West",American Restaurant,Intersection,Motel,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
5,"Dorset Park , Wexford Heights , Scarborough To...",Indian Restaurant,Pet Store,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Smoke Shop,Caribbean Restaurant,General Entertainment,Fried Chicken Joint
6,"Golden Mile , Clairlea , Oakridge",Bakery,Bus Line,Ice Cream Shop,Intersection,Soccer Field,Bus Station,Metro Station,College Stadium,Hakka Restaurant,Grocery Store
7,"Guildwood , Morningside , West Hill",BBQ Joint,Electronics Store,Rental Car Location,Breakfast Spot,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment
8,"Kennedy Park , Ionview , East Birchmount Park",Coffee Shop,Hobby Shop,Department Store,Bus Station,Train Station,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint
9,"Malvern , Rouge",Fast Food Restaurant,Vietnamese Restaurant,Train Station,Hobby Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Department Store


###Run k-means to cluster the neighborhoods into 5 clusters

In [105]:
scarb_data = scarborough_data.drop(16)
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 3], dtype=int32)

###Include kmeans.labels_ into the original Scarborough dataframe

In [106]:
scarb_merged = scarb_data

# add clustering labels
scarb_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarb_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,Train Station,Hobby Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Department Store
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,0,Bar,Vietnamese Restaurant,Coffee Shop,Hobby Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711,0,BBQ Joint,Electronics Store,Rental Car Location,Breakfast Spot,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Ice Cream Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Hakka Restaurant,Bakery,Bank,Fried Chicken Joint,Coffee Shop,Grocery Store,General Entertainment
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0,Playground,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029,0,Coffee Shop,Hobby Shop,Department Store,Bus Station,Train Station,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577,0,Bakery,Bus Line,Ice Cream Shop,Intersection,Soccer Field,Bus Station,Metro Station,College Stadium,Hakka Restaurant,Grocery Store
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476,0,American Restaurant,Intersection,Motel,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848,3,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant


###Visualize the clusters in the map

In [108]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighborhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

###Examine each of the five clusters

In [96]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Vietnamese Restaurant,Train Station,Hobby Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Department Store
1,Scarborough,0,Bar,Vietnamese Restaurant,Coffee Shop,Hobby Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Scarborough,0,BBQ Joint,Electronics Store,Rental Car Location,Breakfast Spot,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment
3,Scarborough,0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Ice Cream Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,Scarborough,0,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Hakka Restaurant,Bakery,Bank,Fried Chicken Joint,Coffee Shop,Grocery Store,General Entertainment
5,Scarborough,0,Playground,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
6,Scarborough,0,Coffee Shop,Hobby Shop,Department Store,Bus Station,Train Station,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint
7,Scarborough,0,Bakery,Bus Line,Ice Cream Shop,Intersection,Soccer Field,Bus Station,Metro Station,College Stadium,Hakka Restaurant,Grocery Store
8,Scarborough,0,American Restaurant,Intersection,Motel,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
13,Scarborough,0,Pizza Place,Noodle House,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Italian Restaurant,Rental Car Location,Pharmacy,Coffee Shop,Chinese Restaurant


In [98]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,1,Middle Eastern Restaurant,Auto Garage,Smoke Shop,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant


In [99]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]    

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,2,Indian Restaurant,Pet Store,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Smoke Shop,Caribbean Restaurant,General Entertainment,Fried Chicken Joint
12,Scarborough,2,Skating Rink,Sandwich Place,Lounge,Breakfast Spot,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant


In [100]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,3,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant


In [101]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Scarborough,4,Fast Food Restaurant,Chinese Restaurant,Noodle House,Breakfast Spot,Japanese Restaurant,Cosmetics Shop,Coffee Shop,Pharmacy,Pizza Place,Grocery Store
